In [1]:
%matplotlib inline
from preamble import *
plt.rcParams['image.cmap'] = "gray"

In [2]:
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import pandas as pd
import smart_open
import random

In [3]:
#인터뷰 text 데이터 불러오기 + 예외처리

from konlpy.corpus import kobill
docs_ko = []
for i in kobill.fileids():
    try:
        docs_ko.append(kobill.open(i).read())
    except UnicodeDecodeError:
        print(i)

In [4]:
docs_ko

['DSJY: \t자 저희는 유니스트 소속 대학원 수업 CONTEXTUAL DESGIN 을 수강하고 있는 이지영입니다. 저희는 학생창업자로서 겪는 여러가지 경험을 자세히 듣고자하며, 특히 업무 진행, 갈등 중재, 활용하는 테크놀로지에 대해 관심이 많아서 이렇게 인터뷰를 진행하게 되었습니다. 인터뷰는 약 30분정도 진행할 것 같고요. 그럼 시작 할꼐요.\nP2\t네\nDSJY\t보나는 어떤 집단인지 궁금합니다\nP2\t우리는 습관에 대해서 항상하고 사람들이 움직이는 방법에 대해서 건강한 습관을 가지고 내가 원하는것, 보나라는 이름이 원래 보고싶은 나라는 뜻이에요. 내가 원하는것과 하고싶어하는 것을 습관을 통해서 이루어간다는 비전을 가지고 있고요. 지금 하는 것은 여러가지 시도를 해보다가 현재는 건강음식 건강음료 이런것을 유니스트의 학생들이 건강하게 살면서 할 수 있도록 지금 준비중에 있습니다.\nDSJY\t굉장히 좋은 취지인것 같습니다\nP2\t그렇죠? 우리팀에서 일할래?\nDSJY\t돈주면..., 보나의 규모가 어느정도인가요?\nP2\t규모라는게 너무 작아서 규모라고 할 정도가 아니고 정말 작아서 그냥 1인기업, 스타트업이라고 생각하시면 될것 같습니다. 풀타임 워커가 3명, 하프타임이 1명, 풀타임이라는것은 9시부터 나와서 9시부터 5시반까지 지키는 사람들 보통은 그 시간 이상까지 있고요. 나머지는 수업이 있고 갔다오고 그러고요. 매출이나 그런게 없다보니 규모로 인원 규모 밖에 말씀을 못드립니다.\nDSJY\t업무환경에 대해서 말씀해 주실수 있나요? 오피스의 컴퓨터라던지?\nP\t2오피스에 컴퓨터 한대 그리고 회의가능하게 화이트보드가 있고, 업무환경의 소프트웨어같은 쪽에서는 잔디,슬랙 이런것 중에서 저희는 잔디를 주로 사용을 하고요. 카톡과는 다른 업무용 메신저를 써서 이제 엄무시간 동안에는 될수 있도록 카톡을 안들어갈려고 하는 중입니다. 주변에서 우리끼리도 잔디를 쓰고 카톡을 키면 일하지 않는 걸로. 매번 안할수는 없겠지만 켜두지 말자는 거죠. 컴퓨터로 일하

In [5]:
def read_corpus(documents):
    for i, plot in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot, max_len=1000), [i])

In [6]:
train_corpus = list(read_corpus(docs_ko))

In [7]:
train_corpus

[TaggedDocument(words=['dsjy', '저희는', '유니스트', '소속', '대학원', '수업', 'contextual', 'desgin', '수강하고', '있는', '이지영입니다', '저희는', '학생창업자로서', '겪는', '여러가지', '경험을', '자세히', '듣고자하며', '특히', '업무', '진행', '갈등', '중재', '활용하는', '테크놀로지에', '대해', '관심이', '많아서', '이렇게', '인터뷰를', '진행하게', '되었습니다', '인터뷰는', '분정도', '진행할', '같고요', '그럼', '시작', '할꼐요', 'dsjy', '보나는', '어떤', '집단인지', '궁금합니다', '우리는', '습관에', '대해서', '항상하고', '사람들이', '움직이는', '방법에', '대해서', '건강한', '습관을', '가지고', '내가', '원하는것', '보나라는', '이름이', '원래', '보고싶은', '나라는', '뜻이에요', '내가', '원하는것과', '하고싶어하는', '것을', '습관을', '통해서', '이루어간다는', '비전을', '가지고', '있고요', '지금', '하는', '것은', '여러가지', '시도를', '해보다가', '현재는', '건강음식', '건강음료', '이런것을', '유니스트의', '학생들이', '건강하게', '살면서', '있도록', '지금', '준비중에', '있습니다', 'dsjy', '굉장히', '좋은', '취지인것', '같습니다', '그렇죠', '우리팀에서', '일할래', 'dsjy', '돈주면', '보나의', '규모가', '어느정도인가요', '규모라는게', '너무', '작아서', '규모라고', '정도가', '아니고', '정말', '작아서', '그냥', '인기업', '스타트업이라고', '생각하시면', '될것', '같습니다', '풀타임', '워커가', '하프타임이', '풀타임이라는것은', '시부터', '나와서', '시부터', '시반까지', '지키는', '사람들', '보통은', '시간', '이상까

In [8]:
train_corpus[:2]

[TaggedDocument(words=['dsjy', '저희는', '유니스트', '소속', '대학원', '수업', 'contextual', 'desgin', '수강하고', '있는', '이지영입니다', '저희는', '학생창업자로서', '겪는', '여러가지', '경험을', '자세히', '듣고자하며', '특히', '업무', '진행', '갈등', '중재', '활용하는', '테크놀로지에', '대해', '관심이', '많아서', '이렇게', '인터뷰를', '진행하게', '되었습니다', '인터뷰는', '분정도', '진행할', '같고요', '그럼', '시작', '할꼐요', 'dsjy', '보나는', '어떤', '집단인지', '궁금합니다', '우리는', '습관에', '대해서', '항상하고', '사람들이', '움직이는', '방법에', '대해서', '건강한', '습관을', '가지고', '내가', '원하는것', '보나라는', '이름이', '원래', '보고싶은', '나라는', '뜻이에요', '내가', '원하는것과', '하고싶어하는', '것을', '습관을', '통해서', '이루어간다는', '비전을', '가지고', '있고요', '지금', '하는', '것은', '여러가지', '시도를', '해보다가', '현재는', '건강음식', '건강음료', '이런것을', '유니스트의', '학생들이', '건강하게', '살면서', '있도록', '지금', '준비중에', '있습니다', 'dsjy', '굉장히', '좋은', '취지인것', '같습니다', '그렇죠', '우리팀에서', '일할래', 'dsjy', '돈주면', '보나의', '규모가', '어느정도인가요', '규모라는게', '너무', '작아서', '규모라고', '정도가', '아니고', '정말', '작아서', '그냥', '인기업', '스타트업이라고', '생각하시면', '될것', '같습니다', '풀타임', '워커가', '하프타임이', '풀타임이라는것은', '시부터', '나와서', '시부터', '시반까지', '지키는', '사람들', '보통은', '시간', '이상까

In [9]:
model = gensim.models.doc2vec.Doc2Vec(size=30, min_count=5, iter=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

1115335

In [10]:
model.save_word2vec_format('doc_tensor.w2v', doctag_vec=True, word_vec=False)

In [11]:
print(model['창업'])

[-0.244  0.354  0.74   0.52   0.081  1.088  0.168 -0.133  0.317  0.65  -0.32
  0.716 -0.357 -0.024  0.063  0.112  0.887  0.086 -0.469  0.884 -0.305
  0.05   0.419  0.096  0.398 -0.075  0.381  0.535  0.191  0.458]


In [12]:
model.most_similar(positive=["의사소통이"])

[('되지', 0.9620218276977539),
 ('어려움을', 0.9543507695198059),
 ('않아서', 0.9474727511405945),
 ('적이', 0.9043252468109131),
 ('겪었던', 0.8757489323616028),
 ('않았던', 0.8715224266052246),
 ('마찰이나', 0.8614959120750427),
 ('팀원간의', 0.8570653200149536),
 ('발생한', 0.8390113115310669),
 ('같은게', 0.8383753299713135)]